In [10]:
import scipy.io as sio
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import numpy.linalg as LA

# SINDy

In [2]:
FILE =sio.loadmat("/Users/wangyuning/Desktop/Data-driven Methods in Engineering Mechanics/SM2001_Project/Data generator (Moehlis model)/moehlis_data_1000.mat")
DATA  = FILE["data"]
print(FILE.keys())
print(f"The shape of whole dataset is {DATA.shape}")

dict_keys(['__header__', '__version__', '__globals__', 'data'])
The shape of whole dataset is (1000, 4000, 9)


In [5]:
# Number of Timeseries
SERIES_NUM = DATA.shape[0]

# Timesteps
TIME = DATA.shape[1]

# ODE
ODE = DATA.shape[2]

In [7]:
def Pool_Data(X,nVars:int,poly:int):


    # X1 = X[:,0]
    # X2 = X[:,1]


    n = X.shape[0]
    m = np.int64(1+nVars+(nVars*(nVars+1)/2)+(nVars*(nVars+1)*(nVars+2)/(2*3))+11)
    yout = np.zeros((n,m))
    ind = 0 
    yout[:,ind] = np.ones((n))
    ind+= 1
    if poly >=1:
        for i in range(nVars):
            yout[:,ind] = X[:,i]
            ind +=1
    if poly >=2:
        for i in range(nVars):
            for j in range(i,nVars):
                yout[:,ind] = X[:,i] * X[:,j]
                ind +=1
    if poly >=3:
        for i in range(nVars):
            for j in range(i,nVars):
                for k in range(j,nVars):
                    yout[:,ind] = X[:,i] * X[:,j] * X[:,k]
                    ind+=1
    return yout

In [11]:
def SparsifyDynamics(Xi,Theta,dX,Lambda,K):
    for k in range(K):
        smallinds = np.abs(Xi) <Lambda
        Xi[smallinds] = 0
        for ind in range(2):
            biginds = ~smallinds[:,ind]
            res = LA.lstsq(Theta[:,biginds],dX[:,ind])
            # print(res)
            Xi[biginds,ind]  = res[0]
    
    return Xi 

In [12]:
A = DATA[0,:,:]
print(A.shape)

(4000, 9)


In [14]:
dA = np.zeros((3999,9))
dt = 1
for i in range(ODE):
    dA[:,i] = (A[1:,i] - A[0:-1,i])/dt


In [22]:
init = np.array([[1, 0.07066, -0.07076, 0, 0, 0, 0, 0, 0]])
print(init.shape)
print(dA.shape)
dA_init = np.vstack([init,dA])
dA_init.shape

(1, 9)
(3999, 9)


(4000, 9)

# 2 Build the Library

In [24]:
X_in = A
Theta = Pool_Data(X_in,nVars=ODE,poly=3)
print(f"Shape of library is {Theta.shape}")

Shape of library is (4000, 231)


In [35]:
import scipy.linalg as LA
Res = LA.lstsq(Theta,dA_init)
Xi = Res[0]
Psi = SparsifyDynamics(Xi,Theta,dA_init,Lambda=0.13,K=10)
print(f"Psi:\n{Psi}\n")
A_recon = Theta @ Psi
print(f"\n{A_recon.shape}\n")
print(f"MAE with dX1: {np.mean(np.abs(A_recon - dA_init))}")
# print(f"MAE with dX2: {np.mean(np.abs(X_recon[:,1] - dX[:,1]))}")

Psi:
[[-0.32465227  0.          0.         ...  0.          0.
   0.        ]
 [ 3.55838417  0.         -0.1915145  ...  0.          0.
   0.        ]
 [-1.0089084   0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


(4000, 9)

MAE with dX1: 0.007870984962723171
